# Simple MNIST convnet

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2015/06/19<br>
**Last modified:** 2020/04/21<br>
**Description:** A simple convnet that achieves ~99% test accuracy on MNIST.

## Setup

In [55]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.decomposition import PCA

## Prepare the data

In [56]:
# Model / data parameters
num_classes = 10
input_shape = 80

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

reshaped_training_data = x_train.ravel().reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])
reshaped_test_data = x_test.ravel().reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2])

print("x_train shape:", reshaped_training_data.shape)
print(reshaped_training_data.shape[0], "train samples")
print(reshaped_test_data.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 784)
60000 train samples
10000 test samples


In [57]:
pca = PCA(n_components=input_shape)
X_pca_train = pca.fit_transform(reshaped_training_data)
X_pca_test = pca.transform(reshaped_test_data)

## Build the model

In [58]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(20, activation="relu"),
        layers.Dense(20, activation="relu"),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 20)                1620      
                                                                 
 dense_13 (Dense)            (None, 20)                420       
                                                                 
 dense_14 (Dense)            (None, 10)                210       
                                                                 
Total params: 2,250
Trainable params: 2,250
Non-trainable params: 0
_________________________________________________________________


## Train the model

In [59]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(X_pca_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/15
469/469 [==============================] - 1s 2ms/step - loss: 0.9742 - accuracy: 0.7013
Epoch 2/15
469/469 [==============================] - 1s 2ms/step - loss: 0.3021 - accuracy: 0.9118
Epoch 3/15
469/469 [==============================] - 1s 2ms/step - loss: 0.2391 - accuracy: 0.9290
Epoch 4/15
469/469 [==============================] - 1s 2ms/step - loss: 0.2091 - accuracy: 0.9380
Epoch 5/15
469/469 [==============================] - 1s 2ms/step - loss: 0.1892 - accuracy: 0.9438
Epoch 6/15
469/469 [==============================] - 1s 2ms/step - loss: 0.1751 - accuracy: 0.9477
Epoch 7/15
469/469 [==============================] - 1s 2ms/step - loss: 0.1645 - accuracy: 0.9510
Epoch 8/15
469/469 [==============================] - 1s 2ms/step - loss: 0.1557 - accuracy: 0.9536
Epoch 9/15
469/469 [==============================] - 1s 2ms/step - loss: 0.1488 - accuracy: 0.9561
Epoch 10/15
469/469 [==============================] - 1s 2ms/step - loss: 0.1430 - accuracy: 0.9575

## Evaluate the trained model

In [60]:
score = model.evaluate(X_pca_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.1414734572172165
Test accuracy: 0.958299994468689


In [61]:
score = model.evaluate(X_pca_train, y_train, verbose=0)
print("Train loss:", score[0])
print("Train accuracy:", score[1])

Train loss: 0.11746692657470703
Train accuracy: 0.9659333229064941


In [62]:
oldWeights = model.get_weights()

In [63]:
weights = []

for layer in oldWeights:
  weights.append(layer.round(2))

model.set_weights(weights)
model.get_weights()[0][0][0]

0.05

In [64]:
score1 = model.evaluate(X_pca_train, y_train, verbose=0)
print("Train loss:", score1[0])
print("Train accuracy:", score1[1])

(score[1]-score1[1])*100

Train loss: 0.1175757497549057
Train accuracy: 0.9659833312034607


-0.0050008296966552734

In [71]:
for layer in model.get_weights():
  print(layer.max())

0.78
0.81
0.87
0.62
1.19
0.52


0.87